In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed

# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/mlcourse-dota2-win-prediction/test_matches.jsonl
/kaggle/input/mlcourse-dota2-win-prediction/test_features.csv
/kaggle/input/mlcourse-dota2-win-prediction/train_matches.jsonl
/kaggle/input/mlcourse-dota2-win-prediction/train_features.csv
/kaggle/input/mlcourse-dota2-win-prediction/train_targets.csv
/kaggle/input/mlcourse-dota2-win-prediction/sample_submission.csv
/kaggle/input/dota-2-datasets/test_features_7_cat.pkl
/kaggle/input/dota-2-datasets/train_features_7_cat.pkl


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from plotly import tools
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import warnings
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import eli5
from sklearn import decomposition
import lightgbm as lgb
import xgboost as xgb

import os
from IPython.display import display_html

import json
from tqdm import tqdm_notebook

/opt/conda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.

Using TensorFlow backend.


In [3]:
X_train = pd.read_pickle("/kaggle/input/dota-2-datasets/train_features_7_cat.pkl")
X_test = pd.read_pickle("/kaggle/input/dota-2-datasets/test_features_7_cat.pkl")

In [4]:
PATH_TO_DATA = '/kaggle/input/mlcourse-dota2-win-prediction/'

In [5]:
df_test_features = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             'test_features.csv'), 
                                    index_col='match_id_hash')

In [6]:
df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                            'train_targets.csv'), 
                                   index_col='match_id_hash')

y = df_train_targets['radiant_win'].values

In [7]:
y = y.astype('float')

In [8]:
columns = list(X_train.columns)
cat_columns = columns[-115:]
cat_columns

['id001',
 'id002',
 'id003',
 'id004',
 'id005',
 'id006',
 'id007',
 'id008',
 'id009',
 'id010',
 'id011',
 'id012',
 'id013',
 'id014',
 'id015',
 'id016',
 'id017',
 'id018',
 'id019',
 'id020',
 'id021',
 'id022',
 'id023',
 'id025',
 'id026',
 'id027',
 'id028',
 'id029',
 'id030',
 'id031',
 'id032',
 'id033',
 'id034',
 'id035',
 'id036',
 'id037',
 'id038',
 'id039',
 'id040',
 'id041',
 'id042',
 'id043',
 'id044',
 'id045',
 'id046',
 'id047',
 'id048',
 'id049',
 'id050',
 'id051',
 'id052',
 'id053',
 'id054',
 'id055',
 'id056',
 'id057',
 'id058',
 'id059',
 'id060',
 'id061',
 'id062',
 'id063',
 'id064',
 'id065',
 'id066',
 'id067',
 'id068',
 'id069',
 'id070',
 'id071',
 'id072',
 'id073',
 'id074',
 'id075',
 'id076',
 'id077',
 'id078',
 'id079',
 'id080',
 'id081',
 'id082',
 'id083',
 'id084',
 'id085',
 'id086',
 'id087',
 'id088',
 'id089',
 'id090',
 'id091',
 'id092',
 'id093',
 'id094',
 'id095',
 'id096',
 'id097',
 'id098',
 'id099',
 'id100',
 'id101',


In [9]:
n_fold = 5
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)

In [10]:
from catboost import CatBoostClassifier
import time

In [11]:
def train_model(X, X_test, y, params, folds, model_type='lgb', plot_feature_importance=False, averaging='usual', model=None):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.loc[train_index], X.loc[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        if model_type == 'lgb':
            train_data = lgb.Dataset(X_train, label=y_train)
            valid_data = lgb.Dataset(X_valid, label=y_valid)
            
            model = lgb.train(params,
                    train_data,
                    num_boost_round=20000,
                    valid_sets = [train_data, valid_data],
                    verbose_eval=1000,
                    early_stopping_rounds = 200)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X_train.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X_train.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X_train.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X_train.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            y_pred_valid = model.predict_proba(X_valid).reshape(-1,)
            score = roc_auc_score(y_valid, y_pred_valid)
            print(f'Fold {fold_n}. AUC: {score:.4f}.')
            print('')
            
            y_pred = model.predict_proba(X_test)[:, 1]
            
        if model_type == 'glm':
            model = sm.GLM(y_train, X_train, family=sm.families.Binomial())
            model_results = model.fit()
            model_results.predict(X_test)
            y_pred_valid = model_results.predict(X_valid).reshape(-1,)
            score = roc_auc_score(y_valid, y_pred_valid)
            
            y_pred = model_results.predict(X_test)
            
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=2500, loss_function='Logloss',  eval_metric='AUC', **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), use_best_model=True, verbose=True, early_stopping_rounds = 200)

            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test)[:, 1]
            
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(roc_auc_score(y_valid, y_pred_valid))

        if averaging == 'usual':
            prediction += y_pred
        elif averaging == 'rank':
            prediction += pd.Series(y_pred).rank().values  
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importance()
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return oof, prediction, feature_importance
        return oof, prediction, scores
    
    else:
        return oof, prediction, scores

In [12]:
params = {
          'cat_features': cat_columns,
          'task_type': 'GPU',
          'verbose': 200,
          'random_seed': 17
         }

In [13]:
X_train[cat_columns] = X_train[cat_columns].astype('str')
X_test[cat_columns] = X_test[cat_columns].astype('str')

In [14]:
%%time

oof_cat, prediction_cat, scores = train_model(X_train, X_test, y, params=params, 
                                              folds=folds, model_type='cat', 
                                              plot_feature_importance=True)

Fold 0 started at Tue Nov 19 18:28:06 2019
Learning rate set to 0.075788
0:	learn: 0.7997876	test: 0.7980486	best: 0.7980486 (0)	total: 241ms	remaining: 10m 2s
1:	learn: 0.8097700	test: 0.8064966	best: 0.8064966 (1)	total: 465ms	remaining: 9m 40s
2:	learn: 0.8124043	test: 0.8104015	best: 0.8104015 (2)	total: 684ms	remaining: 9m 29s
3:	learn: 0.8149047	test: 0.8122222	best: 0.8122222 (3)	total: 907ms	remaining: 9m 25s
4:	learn: 0.8153318	test: 0.8123934	best: 0.8123934 (4)	total: 1.14s	remaining: 9m 26s
5:	learn: 0.8154657	test: 0.8123584	best: 0.8123934 (4)	total: 1.36s	remaining: 9m 24s
6:	learn: 0.8166736	test: 0.8137401	best: 0.8137401 (6)	total: 1.58s	remaining: 9m 21s
7:	learn: 0.8170785	test: 0.8143470	best: 0.8143470 (7)	total: 1.79s	remaining: 9m 18s
8:	learn: 0.8176759	test: 0.8149215	best: 0.8149215 (8)	total: 2.02s	remaining: 9m 20s
9:	learn: 0.8191933	test: 0.8163982	best: 0.8163982 (9)	total: 2.24s	remaining: 9m 18s
10:	learn: 0.8205539	test: 0.8179035	best: 0.8179035 (10)

In [15]:
df_submission = pd.DataFrame(
    {'radiant_win_prob': prediction_cat}, 
    index=df_test_features.index,)

df_submission.to_csv('sumbmission_cat1_f7.csv')